# Heuristics Visualization
This notebook is a demonstration of how the elbow finding heuristics will work on different datasets.
It is intended for experimentation and to determine if any adjustments should be made to handle cases such as an S-curve along the expected linear descent line.

In [ ]:
import numpy as np
import sklearn.metrics
from top2vec.cutoff_heuristics import (
    find_cutoff,
    get_distances_from_line,
    _get_shifted_second_derivative,
    ELBOW_HEURISTIC_STR,
    DERIVATIVE_HEURISTIC_STR,
    AVERAGE_HEURISTIC_STR,
)

%matplotlib inline
import matplotlib.pyplot as plt


def plot_heuristic(
    values,
    figure_num="1",
    figsize=(8, 3),
    first_elbow=True,
    below_line_exclusive=True,
    max_first_delta=0.33
):
    sorted_values = -np.sort(-np.array(values))
    x = np.arange(sorted_values.size)

    m = (sorted_values[-1] - sorted_values[0]) / (sorted_values.size - 1)
    line = x * m + sorted_values[0]

    # Uniform is an absolute value and therefore useless for detecting an inflection
    # y_distances = get_distances_from_line(
    #    sorted_values, m, sorted_values[0], metric="raw-y", first_elbow=False
    # ).distances

    elbow = find_cutoff(
        sorted_values,
        cutoff_heuristic=ELBOW_HEURISTIC_STR,
        first_elbow=first_elbow,
        below_line_exclusive=below_line_exclusive,
        max_first_delta=max_first_delta
    )
    distances_tuple = get_distances_from_line(
        sorted_values, m, sorted_values[0], first_elbow=first_elbow
    )
    y_distances = distances_tuple.y_deltas

    slid_second_derivative = _get_shifted_second_derivative(
        sorted_values, distances_tuple.is_truncated, distances_tuple.truncation_index
    )
    scores = (
        distances_tuple.distances[: distances_tuple.truncation_index + 1]
        * slid_second_derivative
    )
    alt_elbow = find_cutoff(
        sorted_values,
        cutoff_heuristic=DERIVATIVE_HEURISTIC_STR,
        first_elbow=first_elbow,
        below_line_exclusive=below_line_exclusive,
        max_first_delta=max_first_delta
    )
    average_elbow = find_cutoff(
        sorted_values,
        cutoff_heuristic=AVERAGE_HEURISTIC_STR,
        first_elbow=first_elbow,
        below_line_exclusive=below_line_exclusive,
        max_first_delta=max_first_delta
    )
    ELBOW_COLOR = "blue"
    DERIVATIVE_COLOR = "orange"
    AVERGE_COLOR = "green"
    # PLOTS ON PLOTS
    fig = plt.figure(num=figure_num, clear=True, figsize=figsize)

    gs = fig.add_gridspec(nrows=3, ncols=3)
    ax = fig.add_subplot(gs[:2, 0])
    ax.plot(line)
    ax.scatter([elbow], [sorted_values[elbow]], color=ELBOW_COLOR)
    ax.scatter([alt_elbow], [sorted_values[alt_elbow]], color=DERIVATIVE_COLOR)
    ax.scatter([average_elbow], [sorted_values[average_elbow]], color=AVERGE_COLOR)
    ax.plot(sorted_values)

    ax_y = fig.add_subplot(gs[2, 0])
    ax_y.axhline(0, color="black")
    ax_y.plot(y_distances)
    ax_y.scatter([elbow], [y_distances[elbow]], color=ELBOW_COLOR)
    ax_y.scatter([alt_elbow], [y_distances[alt_elbow]], color=DERIVATIVE_COLOR)
    ax_y.scatter([average_elbow], [y_distances[average_elbow]], color=AVERGE_COLOR)

    ax_d = fig.add_subplot(gs[0, 1])
    ax_d.plot(distances_tuple.distances[: distances_tuple.truncation_index + 1])
    ax_d.axhline(0, color="black")
    ax_d.scatter([elbow], [distances_tuple.distances[elbow]], color=ELBOW_COLOR)
    ax_d.xaxis.set_ticklabels([])
    ax_val_second_d = fig.add_subplot(gs[1, 1])
    ax_val_second_d.plot(slid_second_derivative)
    ax_val_second_d.scatter(
        [alt_elbow], [slid_second_derivative[alt_elbow]], color=DERIVATIVE_COLOR
    )
    ax_val_second_d.axhline(0, color="black")
    ax_val_scores = fig.add_subplot(gs[2, 1])
    ax_val_scores.plot(scores)
    ax_val_scores.scatter([alt_elbow], [scores[alt_elbow]], color=DERIVATIVE_COLOR)
    ax_val_scores.axhline(0, color="black")

## Manufactured Data

In [ ]:
vector = np.array([[2, 1]])
base = np.array(
    [
        [0, 1],
        [2, 1],
        [1, 0.5],
        [1, 0],
        [-1, 2],
    ]
)
opposite = np.array(
    [
        [-2, -0.8],
        [-2, -0.9],
        [-2, -0.95],
        [-2, -1],
        [-2, -1.05],
        [-2, -1.1],
        [-2, -1.2],
        [-2, -1.5],
    ]
)

test_embedding = np.vstack(
    [
        np.array(
            [
                [4, 2],
                [8, 4],
                [1, -1],
            ]
        ),
        base,
    ]
)

test_embedding_multiple_elbows = np.vstack([test_embedding, opposite])

# A different divergent case where the S-curve goes the other way
test_embedding_multiple_elbows_2 = np.vstack(
    [base, opposite, np.full((10, 2), [1, -1])]
)

sims = 1 - sklearn.metrics.pairwise_distances(vector, test_embedding, metric="cosine")
sims_multiple_elbows = 1 - sklearn.metrics.pairwise_distances(
    vector, test_embedding_multiple_elbows, metric="cosine"
)
sims_multiple_elbows_2 = 1 - sklearn.metrics.pairwise_distances(
    vector, test_embedding_multiple_elbows_2, metric="cosine"
)

## Show heuristic performance on simple embedding - one elbow

In [ ]:
# The current elbow-finding heuristic also showing changes in distance from the line
plot_heuristic(
    sims[0],
    figure_num="Basic - Distance",
    figsize=(18, 8),
    first_elbow=False,
)

## Showing heuristic performance with multiple elbows in graph

In [ ]:
# The current elbow-finding heuristic also showing changes in distance from the line
plot_heuristic(
    np.hstack((sims_multiple_elbows[0], [-1.5])),
    figure_num="2 Elbows - Exclusive",
    figsize=(18, 8),
    first_elbow=False,
    below_line_exclusive=True,
)
plot_heuristic(
    np.hstack((sims_multiple_elbows[0], [-1.5])),
    figure_num="2 Elbows - Inclusive",
    figsize=(18, 8),
    first_elbow=False,
    below_line_exclusive=False,
)
plot_heuristic(
    np.hstack((sims_multiple_elbows[0], [-1.5])),
    figure_num="2 Elbows - First Elbow",
    figsize=(18, 8),
    first_elbow=True,
)

## Yet another form of a graph with multiple elbows

In [ ]:
# The current elbow-finding heuristic also showing changes in distance from the line
plot_heuristic(
    sims_multiple_elbows_2[0],
    figure_num="3 Elbows",
    figsize=(18, 8),
    first_elbow=False,
)
plot_heuristic(
    sims_multiple_elbows_2[0],
    figure_num="3 Elbows - First Elbow",
    figsize=(18, 8),
    first_elbow=True,
)

## Data from online

In [ ]:
# This is an example of running only on the stuff which was already determined to be similar
# but in this case it appears that we want our elbow to be exclusive, not inclusive

sample_data = np.array(
    [
        0.87387407,
        0.8490747,
        0.83483994,
        0.80989516,
        0.45845926,
        0.45052826,
        0.44408453,
        0.4278804,
        0.4249642,
        0.41800153,
        0.415339,
        0.40166456,
        0.4011852,
        0.3939832,
        0.38374978,
        0.3823452,
        0.37897837,
        0.37643087,
        0.37551993,
        0.37453377,
        0.37433827,
        0.3703115,
        0.36441594,
        0.3591773,
        0.35516483,
        0.35450447,
        0.35071152,
        0.34965813,
        0.3412869,
        0.3399775,
        0.33868152,
        0.33400196,
        0.315881,
    ]
)

plot_heuristic(
    sample_data,
    figure_num="Online data",
    figsize=(18, 8),
    first_elbow=True,
)

## Speed Tests

In [ ]:
%timeit -n 100000 -r 10 elbows = find_cutoff(sims_multiple_elbows_2[0], cutoff_heuristic=ELBOW_HEURISTIC_STR, first_elbow=False)
%timeit -n 100000 -r 10 elbows = find_cutoff(sims_multiple_elbows_2[0], cutoff_heuristic=ELBOW_HEURISTIC_STR, first_elbow=True)

In [ ]:
# What if it is the same shape but much larger?
print(sims_multiple_elbows_2.shape)
big_sims = np.tile(sims_multiple_elbows_2, 1000000)
print(big_sims.shape)

In [ ]:
%timeit -n 10 -r 5  elbows = find_cutoff(big_sims[0], cutoff_heuristic=ELBOW_HEURISTIC_STR, first_elbow=False)
%timeit -n 10 -r 5  elbows = find_cutoff(big_sims[0], cutoff_heuristic=ELBOW_HEURISTIC_STR, first_elbow=True)

### Performance Metrics
This has 3 elbows so we expect to have a substantial performance boost from using first_elbow

#### Size 23
17.4 µs ± 505 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)  
18.9 µs ± 428 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)  


#### Size 230,000
5.43 ms ± 12.6 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)  
5.47 ms ± 31.2 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)  

#### Size 2,300,000
57.7 ms ± 845 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)  
58.1 ms ± 157 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)  

#### Size 23,000,000
632 ms ± 1.5 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)  
649 ms ± 573 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)  